In [1]:
import numpy as np
from scipy import sparse, stats

In [33]:
rate_df = pd.DataFrame([[1, 3, 0, 3], [0, 1, 3, 0], [2, 1, 3, 1], [1, 3, 2, 0]])
rate_df

,0,1,2,3
0,1,3,0,3
1,0,1,3,0
2,2,1,3,1
3,1,3,2,0


In [110]:
(n_users, n_items) = ratings.shape

user_ids = rate_df.index
user_id2idx = {user_id: i for i, user_id in enumerate(user_ids)}

item_ids = rate_df.columns
item_id2idx = {item_id: i for i, item_id in enumerate(item_ids)}

ratings = rate_df.values

In [59]:
def get_common_indices(x: np.array, y: np.array) -> np.array:
    return np.intersect1d(np.where(x > 0)[0], np.where(y > 0)[0])

In [197]:
p = sparse.lil_matrix((n_users, n_users))
for i in user_ids:
    x = ratings[i]

    for j in user_ids:
        if i == j:
            p[i, j] = 0
            continue

        y = ratings[j]
        common_evaluated_items = get_common_indices(x, y)
        if common_evaluated_items.shape[0] < 2:
            p[i, j] = 0
            continue

        p[i, j] = stats.pearsonr(
            x[common_evaluated_items],
            y[common_evaluated_items]
        )[0]

r_means = sparse.lil_matrix((n_users, n_users))
for i in user_ids:
    x = ratings[i]

    for j in user_ids:
        y = ratings[j]
        common_evaluated_items = get_common_indices(x, y)
        r_means[i, j] = np.mean(x[common_evaluated_items])

r = sparse.lil_matrix((n_users, n_items))
for i in user_ids:
    x = ratings[i]

    not_evaluated_item_ids = np.where(x == 0.0)[0]
    for j in not_evaluated_item_ids:
        evaluated_user_ids = np.where(ratings.T[j] > 0.0)[0]
        
        p_sum = np.sum(np.absolute(p[i, evaluated_user_ids]))
        r_sum = np.sum([p[i, k] * (ratings[k, j] - r_means[k, i]) for k in evaluated_user_ids if r_means[k, i] > 0.0])
        
        r[i, j] = r_means[i, i] + r_sum / p_sum
r

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in LInked List format>

In [193]:
ratings

array([[1, 3, 0, 3],
       [0, 1, 3, 0],
       [2, 1, 3, 1],
       [1, 3, 2, 0]])

In [194]:
r_means.toarray()

array([[2.33333333, 3.        , 2.33333333, 2.        ],
       [1.        , 2.        , 2.        , 2.        ],
       [1.33333333, 2.        , 1.75      , 2.        ],
       [2.        , 2.5       , 2.        , 2.        ]])

In [195]:
p.toarray()

array([[ 0. ,  0. , -1. ,  1. ],
       [ 0. ,  0. ,  1. , -1. ],
       [-1. ,  1. ,  0. , -0.5],
       [ 1. , -1. , -0.5,  0. ]])

In [196]:
r.toarray()

array([[0.  , 0.  , 1.5 , 0.  ],
       [2.75, 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 3.  ]])